# <center>MobileNet - Pytorch

# Step 1: Prepare data

In [1]:
# MobileNet-Pytorch
import argparse 
import torch
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR
from torchvision import datasets, transforms
from torch.autograd import Variable
from torch.utils.data.sampler import SubsetRandomSampler
from sklearn.metrics import accuracy_score
from mobilenets import mobilenet

use_cuda = torch.cuda.is_available()
use_cudause_cud  = torch.cuda.is_available()
dtype = torch.cuda.FloatTensor if use_cuda else torch.FloatTensor

In [2]:
# Train, Validate, Test. Heavily inspired by Kevinzakka https://github.com/kevinzakka/DenseNet/blob/master/data_loader.py

normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])

valid_size=0.1

# define transforms
valid_transform = transforms.Compose([
        transforms.ToTensor(),
        normalize
])

train_transform = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    normalize
])


# load the dataset
train_dataset = datasets.CIFAR10(root="data", train=True, 
            download=True, transform=train_transform)

valid_dataset = datasets.CIFAR10(root="data", train=True, 
            download=True, transform=valid_transform)

num_train = len(train_dataset)
indices = list(range(num_train))
split = int(np.floor(valid_size * num_train)) #5w张图片的10%用来当做验证集


np.random.seed(42)
np.random.shuffle(indices) # 随机乱序[0,1,...,49999]

train_idx, valid_idx = indices[split:], indices[:split]


train_sampler = SubsetRandomSampler(train_idx) # 这个很有意思
valid_sampler = SubsetRandomSampler(valid_idx)


# ------------------------- 使用不同的批次大小 ------------------------------------
show_step=20  # 批次大，show_step就小点
max_epoch=50  # 训练最大epoch数目
train_loader = torch.utils.data.DataLoader(train_dataset, 
                batch_size=256, sampler=train_sampler)

valid_loader = torch.utils.data.DataLoader(valid_dataset, 
                batch_size=256, sampler=valid_sampler)


test_transform = transforms.Compose([
    transforms.ToTensor(), normalize
])

test_dataset = datasets.CIFAR10(root="data", 
                                train=False, 
                                download=True,transform=test_transform)

test_loader = torch.utils.data.DataLoader(test_dataset, 
                                          batch_size=64, 
                                          shuffle=True)

Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified


# Step 2: Model Config

In [5]:
# 32  缩放5次到 1x1@1024 
# From https://github.com/kuangliu/pytorch-cifar 
import torch
import torch.nn as nn
import torch.nn.functional as F


class Block(nn.Module):
    '''Depthwise conv + Pointwise conv'''
    def __init__(self, in_planes, out_planes, stride=1):
        super(Block, self).__init__()
        
        # 分组卷积数=输入通道数
        self.conv1 = nn.Conv2d(in_planes, in_planes, kernel_size=3, stride=stride, padding=1, groups=in_planes, bias=False)
        
        self.bn1 = nn.BatchNorm2d(in_planes)
        
        self.conv2 = nn.Conv2d(in_planes, out_planes, kernel_size=1, stride=1, padding=0, bias=False)
        
        self.bn2 = nn.BatchNorm2d(out_planes)

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = F.relu(self.bn2(self.conv2(out)))
        return out


class MobileNet(nn.Module):
    # (128,2) means conv planes=128, conv stride=2, by default conv stride=1
    cfg = [64, (128,2), 128, (256,2), 256, (512,2), 512, 512, 512, 512, 512, (1024,2), 1024]

    def __init__(self, num_classes=10):
        super(MobileNet, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1, bias=False)
        
        self.bn1 = nn.BatchNorm2d(32)
        self.layers = self._make_layers(in_planes=32) # 自动化构建层
        self.linear = nn.Linear(1024, num_classes)

    def _make_layers(self, in_planes):
        layers = []
        for x in self.cfg:
            out_planes = x if isinstance(x, int) else x[0]
            stride = 1 if isinstance(x, int) else x[1]
            layers.append(Block(in_planes, out_planes, stride))
            in_planes = out_planes
        return nn.Sequential(*layers)

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.layers(out)
        out = F.avg_pool2d(out, 2)
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        return out

In [6]:
# From https://github.com/Z0m6ie/CIFAR-10_PyTorch
#model = mobilenet(num_classes=10, large_img=False)

# From https://github.com/kuangliu/pytorch-cifar 
model=MobileNet(10).cuda()

optimizer = optim.Adam(model.parameters(), lr=0.01)
scheduler = StepLR(optimizer, step_size=10, gamma=0.5)
criterion = nn.CrossEntropyLoss()

In [7]:
# Implement validation
def train(epoch):
    model.train()
    #writer = SummaryWriter()
    for batch_idx, (data, target) in enumerate(train_loader):
        if use_cuda:
            data, target = data.cuda(), target.cuda()
        data, target = Variable(data), Variable(target)
        optimizer.zero_grad()
        output = model(data)
        correct = 0
        pred = output.data.max(1, keepdim=True)[1] # get the index of the max log-probability
        correct += pred.eq(target.data.view_as(pred)).sum()
        
        loss = criterion(output, target)
        loss.backward()
        accuracy = 100. * (correct.cpu().numpy()/ len(output))
        optimizer.step()
        if batch_idx % show_step == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}, Accuracy: {:.2f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item(), accuracy))
            #writer.add_scalar('Loss/Loss', loss.item(), epoch)
            #writer.add_scalar('Accuracy/Accuracy', accuracy, epoch)
    scheduler.step()

In [8]:
def validate(epoch):
    model.eval()
    #writer = SummaryWriter()
    valid_loss = 0
    correct = 0
    for data, target in valid_loader:
        if use_cuda:
            data, target = data.cuda(), target.cuda()
        data, target = Variable(data), Variable(target)
        output = model(data)
        valid_loss += F.cross_entropy(output, target, size_average=False).item() # sum up batch loss
        pred = output.data.max(1, keepdim=True)[1] # get the index of the max log-probability
        correct += pred.eq(target.data.view_as(pred)).sum()

    valid_loss /= len(valid_idx)
    accuracy = 100. * correct.cpu().numpy() / len(valid_idx)
    print('\nValidation set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        valid_loss, correct, len(valid_idx),
        100. * correct / len(valid_idx)))
    #writer.add_scalar('Loss/Validation_Loss', valid_loss, epoch)
    #writer.add_scalar('Accuracy/Validation_Accuracy', accuracy, epoch)
    return valid_loss, accuracy

In [9]:
# Fix best model

def test(epoch):
    model.eval()
    test_loss = 0
    correct = 0
    for data, target in test_loader:
        if use_cuda:
            data, target = data.cuda(), target.cuda()
        data, target = Variable(data), Variable(target)
        output = model(data)
        test_loss += F.cross_entropy(output, target, size_average=False).item() # sum up batch loss
        pred = output.data.max(1, keepdim=True)[1] # get the index of the max log-probability
        correct += pred.eq(target.data.view_as(pred)).cpu().sum()

    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct.cpu().numpy() / len(test_loader.dataset)))

In [10]:
def save_best(loss, accuracy, best_loss, best_acc):
    if best_loss == None:
        best_loss = loss
        best_acc = accuracy
        file = 'saved_models/best_save_model.p'
        torch.save(model.state_dict(), file)
        
    elif loss < best_loss and accuracy > best_acc:
        best_loss = loss
        best_acc = accuracy
        file = 'saved_models/best_save_model.p'
        torch.save(model.state_dict(), file)
    return best_loss, best_acc

In [11]:
# Fantastic logger for tensorboard and pytorch, 
# run tensorboard by opening a new terminal and run "tensorboard --logdir runs"
# open tensorboard at http://localhost:6006/
from tensorboardX import SummaryWriter
best_loss = None
best_acc = None

import time 
SINCE=time.time()

for epoch in range(max_epoch):
    train(epoch)
    loss, accuracy = validate(epoch)
    best_loss, best_acc = save_best(loss, accuracy, best_loss, best_acc)
    
    NOW=time.time() 
    DURINGS=NOW-SINCE
    SINCE=NOW
    print("the time of this epoch:[{} s]".format(DURINGS))
    
#writer = SummaryWriter()
#writer.export_scalars_to_json("./all_scalars.json")

#writer.close()

Train Epoch: 0 [0/50000 (0%)]	Loss: 2.308716, Accuracy: 8.59
Train Epoch: 0 [5120/50000 (11%)]	Loss: 1.952591, Accuracy: 18.75
Train Epoch: 0 [10240/50000 (23%)]	Loss: 1.874141, Accuracy: 30.86
Train Epoch: 0 [15360/50000 (34%)]	Loss: 1.855571, Accuracy: 35.55
Train Epoch: 0 [20480/50000 (45%)]	Loss: 1.763193, Accuracy: 30.47
Train Epoch: 0 [25600/50000 (57%)]	Loss: 1.641412, Accuracy: 41.41
Train Epoch: 0 [30720/50000 (68%)]	Loss: 1.526574, Accuracy: 43.36
Train Epoch: 0 [35840/50000 (80%)]	Loss: 1.590946, Accuracy: 43.75
Train Epoch: 0 [40960/50000 (91%)]	Loss: 1.476862, Accuracy: 46.48

Validation set: Average loss: 1.8172, Accuracy: 2090/5000 (41.00%)

the time of this epoch:[19.475614070892334 s]
Train Epoch: 1 [0/50000 (0%)]	Loss: 1.355428, Accuracy: 49.22
Train Epoch: 1 [5120/50000 (11%)]	Loss: 1.587818, Accuracy: 43.36
Train Epoch: 1 [10240/50000 (23%)]	Loss: 1.402530, Accuracy: 51.56
Train Epoch: 1 [15360/50000 (34%)]	Loss: 1.306758, Accuracy: 50.00
Train Epoch: 1 [20480/50000

Train Epoch: 11 [30720/50000 (68%)]	Loss: 0.395524, Accuracy: 88.28
Train Epoch: 11 [35840/50000 (80%)]	Loss: 0.403081, Accuracy: 84.38
Train Epoch: 11 [40960/50000 (91%)]	Loss: 0.478091, Accuracy: 82.81

Validation set: Average loss: 0.4636, Accuracy: 4214/5000 (84.00%)

the time of this epoch:[19.611411094665527 s]
Train Epoch: 12 [0/50000 (0%)]	Loss: 0.347412, Accuracy: 88.67
Train Epoch: 12 [5120/50000 (11%)]	Loss: 0.472834, Accuracy: 83.98
Train Epoch: 12 [10240/50000 (23%)]	Loss: 0.380250, Accuracy: 87.50
Train Epoch: 12 [15360/50000 (34%)]	Loss: 0.346975, Accuracy: 87.11
Train Epoch: 12 [20480/50000 (45%)]	Loss: 0.386466, Accuracy: 85.16
Train Epoch: 12 [25600/50000 (57%)]	Loss: 0.491557, Accuracy: 84.38
Train Epoch: 12 [30720/50000 (68%)]	Loss: 0.363731, Accuracy: 89.45
Train Epoch: 12 [35840/50000 (80%)]	Loss: 0.338835, Accuracy: 89.06
Train Epoch: 12 [40960/50000 (91%)]	Loss: 0.348502, Accuracy: 87.50

Validation set: Average loss: 0.4264, Accuracy: 4267/5000 (85.00%)

the ti

Train Epoch: 23 [5120/50000 (11%)]	Loss: 0.270018, Accuracy: 89.45
Train Epoch: 23 [10240/50000 (23%)]	Loss: 0.234289, Accuracy: 91.41
Train Epoch: 23 [15360/50000 (34%)]	Loss: 0.243568, Accuracy: 91.80
Train Epoch: 23 [20480/50000 (45%)]	Loss: 0.203799, Accuracy: 94.14
Train Epoch: 23 [25600/50000 (57%)]	Loss: 0.203761, Accuracy: 91.02
Train Epoch: 23 [30720/50000 (68%)]	Loss: 0.257220, Accuracy: 91.02
Train Epoch: 23 [35840/50000 (80%)]	Loss: 0.294573, Accuracy: 89.84
Train Epoch: 23 [40960/50000 (91%)]	Loss: 0.227648, Accuracy: 92.19

Validation set: Average loss: 0.3718, Accuracy: 4397/5000 (87.00%)

the time of this epoch:[19.8292133808136 s]
Train Epoch: 24 [0/50000 (0%)]	Loss: 0.205717, Accuracy: 93.36
Train Epoch: 24 [5120/50000 (11%)]	Loss: 0.240026, Accuracy: 91.80
Train Epoch: 24 [10240/50000 (23%)]	Loss: 0.167228, Accuracy: 94.92
Train Epoch: 24 [15360/50000 (34%)]	Loss: 0.124064, Accuracy: 95.31
Train Epoch: 24 [20480/50000 (45%)]	Loss: 0.185969, Accuracy: 92.58
Train Epoc

Train Epoch: 34 [25600/50000 (57%)]	Loss: 0.073120, Accuracy: 96.88
Train Epoch: 34 [30720/50000 (68%)]	Loss: 0.182005, Accuracy: 94.14
Train Epoch: 34 [35840/50000 (80%)]	Loss: 0.116506, Accuracy: 96.09
Train Epoch: 34 [40960/50000 (91%)]	Loss: 0.127928, Accuracy: 95.31

Validation set: Average loss: 0.3677, Accuracy: 4435/5000 (88.00%)

the time of this epoch:[19.7981960773468 s]
Train Epoch: 35 [0/50000 (0%)]	Loss: 0.117623, Accuracy: 95.70
Train Epoch: 35 [5120/50000 (11%)]	Loss: 0.085784, Accuracy: 97.27
Train Epoch: 35 [10240/50000 (23%)]	Loss: 0.157082, Accuracy: 93.36
Train Epoch: 35 [15360/50000 (34%)]	Loss: 0.164775, Accuracy: 93.36
Train Epoch: 35 [20480/50000 (45%)]	Loss: 0.121003, Accuracy: 95.70
Train Epoch: 35 [25600/50000 (57%)]	Loss: 0.129254, Accuracy: 94.92
Train Epoch: 35 [30720/50000 (68%)]	Loss: 0.112294, Accuracy: 94.92
Train Epoch: 35 [35840/50000 (80%)]	Loss: 0.086901, Accuracy: 97.27
Train Epoch: 35 [40960/50000 (91%)]	Loss: 0.148964, Accuracy: 94.92

Validati


Validation set: Average loss: 0.4292, Accuracy: 4467/5000 (89.00%)

the time of this epoch:[19.828170776367188 s]
Train Epoch: 46 [0/50000 (0%)]	Loss: 0.078808, Accuracy: 96.48
Train Epoch: 46 [5120/50000 (11%)]	Loss: 0.090777, Accuracy: 96.09
Train Epoch: 46 [10240/50000 (23%)]	Loss: 0.084993, Accuracy: 97.66
Train Epoch: 46 [15360/50000 (34%)]	Loss: 0.058914, Accuracy: 97.66
Train Epoch: 46 [20480/50000 (45%)]	Loss: 0.090326, Accuracy: 96.88
Train Epoch: 46 [25600/50000 (57%)]	Loss: 0.094977, Accuracy: 96.88
Train Epoch: 46 [30720/50000 (68%)]	Loss: 0.098054, Accuracy: 96.09
Train Epoch: 46 [35840/50000 (80%)]	Loss: 0.071499, Accuracy: 97.27
Train Epoch: 46 [40960/50000 (91%)]	Loss: 0.091228, Accuracy: 96.48

Validation set: Average loss: 0.4183, Accuracy: 4469/5000 (89.00%)

the time of this epoch:[19.58849549293518 s]
Train Epoch: 47 [0/50000 (0%)]	Loss: 0.054134, Accuracy: 98.05
Train Epoch: 47 [5120/50000 (11%)]	Loss: 0.104741, Accuracy: 97.66
Train Epoch: 47 [10240/50000 (23%)]

# Step 3: Test

In [12]:
test(epoch)


Test set: Average loss: 0.4542, Accuracy: 8932/10000 (89.32%)

